In [ ]:
# This code will convert the xml annotations into the binary image of same size as original image

In [17]:
## Load Libraries
%matplotlib inline
from openslide import open_slide
from openslide.deepzoom import DeepZoomGenerator
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
import os
import tifffile as tiff


In [44]:
## Total Number of tiles generated at all the levels
image = open_slide("/blue/pinaki.sarder/akshitagupta/MicrovascularInf/Model_Development/Training/Data/S08-28411 - 2015-06-29 11.19.06.ndpi")
tiles = DeepZoomGenerator(image,tile_size =512,overlap=0,limit_bounds=False)
print('size:', image.level_dimensions[2])
print('Total no of tile:', image.dimensions)
print('tile level:',tiles.level_count)
print('tile level something:', tiles.level_tiles)
print('tile coordinates:', tiles.get_tile_coordinates)

size: (9920, 18816)
Total no of tile: (39680, 75264)
tile level: 18
tile level something: ((1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 2), (2, 3), (3, 5), (5, 10), (10, 19), (20, 37), (39, 74), (78, 147))
tile coordinates: <bound method DeepZoomGenerator.get_tile_coordinates of DeepZoomGenerator(OpenSlide('/blue/pinaki.sarder/akshitagupta/MicrovascularInf/Model_Development/Training/Data/S08-28411 - 2015-06-29 11.19.06.ndpi'), tile_size=512, overlap=0, limit_bounds=False)>


In [19]:
## Read the xml file and convert the polygons into the binary map
import xml.etree.ElementTree as ET
import math
from PIL import Image, ImageDraw

tree = None

In [ ]:
if tree == None:
    tree = ET.parse("/blue/pinaki.sarder/akshitagupta/MicrovascularInf/Model_Development/Training/S15-56749CD34LCA.xml")
root = tree.getroot()
print(root)
img_size = image.level_dimensions[2]
print(img_size)
im = np.zeros((img_size[1],img_size[0]),dtype=np.uint8)
im = np.array(im)
cv.imwrite("your_file.jpeg",im)

## Iterate over the xml files
for Annotation in root.findall("./Annotation"): # for all annotations
    annotationID = Annotation.attrib['Id']
    #print(annotationID)
    for Region in Annotation.findall("./Regions/Region"): # iterate on all region
        regionID = Region.attrib['Id']
        print(regionID)
        points = []
        for Vertex in Region.findall("./Vertices/Vertex"):
            points.append([int(float(Vertex.attrib['X'])/factor),int(float(Vertex.attrib['Y'])/factor)])
        points = np.array(points)    
        #print(points)
        cv.fillConvexPoly(im, np.array(points, 'int32'), 255)
mask = Image.fromarray(im)
mask.save("your_file.jpeg")
#print(mask.size)
#cv.imwrite('mask.tiff',im)


In [ ]:
### Extract patches from the binary mask and same mask from the WSI

In [7]:
#%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
import os
import openslide
import xml.etree.cElementTree as ET
import random
import os 
import glob
from PIL import Image

In [8]:
slidepath = "/blue/pinaki.sarder/akshitagupta/MicrovascularInf/Model_Development/Data_PTC"  #path to folder of WSIs
annotpath = "/blue/pinaki.sarder/akshitagupta/MicrovascularInf/Model_Development/Data_PTC" 
slidename = "S10-69161CD34LCA.ndpi"
annotname = "S10-69161CD34LCA.xml"

In [9]:
mag_level = 2               # here I am reading the WSI at level 2
factor = 2**mag_level

In [10]:
def parse_xml(im,anno_path):
    tree = ET.ElementTree(file=anno_path)
    annolist={}
    root = tree.getroot()
    i = 0
    mag_level = 2               # here I am reading the WSI at level 2
    factor = 2**mag_level
    for Annotation in root.findall("./Annotation"):
        annotationID = Annotation.attrib['Id']
        for Region in Annotation.findall("./Regions/Region"): # iterate on all region
            regionID = Region.attrib['Id']
            #print(regionID)
            vasc = []
            for coord in Region.findall("./Vertices/Vertex"):
                vasc.append([int(float(coord.attrib.get("X"))/factor),int(float(coord.attrib.get("Y"))/factor)])
            annolist[i] = vasc
            i+=1
            points = np.array(vasc)   
            #print(points)
            cv.fillConvexPoly(im, np.array(points, 'int32'), 255)
    #im = np.asarray(im)
    cv.imwrite("mask.jpeg",im)
    return (annolist,im)

In [11]:
def generate_patches(slidepath,slidename,annotpath, annotname, patch_size):
    slide_path = os.path.join(slidepath, slidename)
    anno_path = os.path.join(annotpath,annotname)
    Slide = openslide.OpenSlide(slide_path)
    img_size = Slide.level_dimensions[2]
    print(img_size)
    im = np.zeros((img_size[1],img_size[0]),dtype=np.uint8)
    #im = (Image.new('1', (img_size[1], img_size[0])))
    im = np.array(im)
    print(im.shape)

    annolist, mask = parse_xml(im,anno_path)
    print(mask)
    #print(mask.size)
    #print(annolist) 
    for i in range(len(annolist)):                   
        coords = np.array(annolist[i])
        #print(coords)
        x, y, w, h = cv.boundingRect(coords)
        #print(x,y,w,h)
        for k in range(20):                        
            offsetx = random.randrange(-100, 100)
            offsety = random.randrange(-100, 100)
            spointx, spointy = x + offsetx, y + offsety 
            #top left coords of patch bounding box in mlevel
            spointx0, spointy0 = spointx*factor, spointy*factor #multipled by factor to get the original co-ordinates as in WSI and not the as per the level 
            patchimg = Slide.read_region((spointx0, spointy0), 2, (patch_size, patch_size))
            #print(spointx0, spointy0)
            #cv2.rectangle(imgResize,([spointx0,spointx0+patch_size),(spointy0,spointy0+patch_size),(255,0,0),2)
            #roi = mask[10:1000, 10:1000]
            roi = mask[spointy:spointy+patch_size,spointx:spointx+patch_size]
            cv.imwrite(f"/blue/pinaki.sarder/akshitagupta/MicrovascularInf/Model_Development/Training_PTC/masks/ptc_1_{str(i)}{str(k)}.png", np.array(roi))
            cv.imwrite(f"/blue/pinaki.sarder/akshitagupta/MicrovascularInf/Model_Development/Training_PTC/images/ptc_1_{str(i)}{str(k)}.png", np.array(patchimg))
            #print('hey thr!! Almost got it')

In [12]:
generate_patches(slidepath,slidename,annotpath, annotname, 512)

(20480, 12544)
(12544, 20480)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
